In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [5]:
df_monday=pd.read_csv('./data/weekday/monday.csv', sep=';', parse_dates=['timestamp'])
df_monday

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,3,dairy
3,2019-09-02 07:04:00,4,dairy
4,2019-09-02 07:04:00,5,spices
...,...,...,...
4879,2019-09-02 21:49:00,1442,checkout
4880,2019-09-02 21:49:00,1444,checkout
4881,2019-09-02 21:49:00,1445,dairy
4882,2019-09-02 21:50:00,1446,dairy


### Calculate the total number of customers in each section

In [6]:
df_monday.groupby(['location']).count()

,timestamp,customer_no
location,,
checkout,1437,1437
dairy,895,895
drinks,797,797
fruit,1005,1005
spices,750,750


### Calculate the total number of customers in each section over time

In [7]:
df_monday.groupby('timestamp')['location'].value_counts().unstack().fillna(0)

location,checkout,dairy,drinks,fruit,spices
timestamp,,,,,
2019-09-02 07:03:00,0.0,2.0,0.0,0.0,0.0
2019-09-02 07:04:00,0.0,2.0,0.0,1.0,3.0
2019-09-02 07:05:00,2.0,1.0,1.0,0.0,0.0
2019-09-02 07:06:00,2.0,2.0,1.0,2.0,1.0
2019-09-02 07:07:00,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...
2019-09-02 21:46:00,1.0,0.0,2.0,1.0,2.0
2019-09-02 21:47:00,2.0,2.0,0.0,2.0,0.0
2019-09-02 21:48:00,2.0,2.0,2.0,0.0,1.0


### Calculate the time each customer spent in the market

In [9]:
df_sorted = df_monday.sort_values(by=['customer_no','timestamp'])
df_sorted['time_diff'] = df_sorted.groupby('customer_no')['timestamp'].diff()
df_sorted

,timestamp,customer_no,location,time_diff
0,2019-09-02 07:03:00,1,dairy,NaT
8,2019-09-02 07:05:00,1,checkout,0 days 00:02:00
1,2019-09-02 07:03:00,2,dairy,NaT
12,2019-09-02 07:06:00,2,checkout,0 days 00:03:00
2,2019-09-02 07:04:00,3,dairy,NaT
...,...,...,...,...
4874,2019-09-02 21:48:00,1444,spices,NaT
4880,2019-09-02 21:49:00,1444,checkout,0 days 00:01:00
4881,2019-09-02 21:49:00,1445,dairy,NaT
4882,2019-09-02 21:50:00,1446,dairy,NaT


In [10]:
df_sorted['total_spend_time']=df_sorted['time_diff'].dt.seconds
df_sorted.groupby('customer_no')['total_spend_time'].sum()

customer_no
1       120.0
2       180.0
3       120.0
4       240.0
5        60.0
        ...  
1443     60.0
1444     60.0
1445      0.0
1446      0.0
1447      0.0
Name: total_spend_time, Length: 1447, dtype: float64

#### Calculate the total number of customers in the supermarket over time


In [11]:
df_sorted.groupby('timestamp')['customer_no'].nunique()

timestamp
2019-09-02 07:03:00    2
2019-09-02 07:04:00    6
2019-09-02 07:05:00    4
2019-09-02 07:06:00    8
2019-09-02 07:07:00    4
                      ..
2019-09-02 21:46:00    6
2019-09-02 21:47:00    6
2019-09-02 21:48:00    7
2019-09-02 21:49:00    7
2019-09-02 21:50:00    2
Name: customer_no, Length: 874, dtype: int64

### Markov Chains - Transition Matrix

In [13]:
#Plot the distribution of customers of their first visited section versus following sections
df_sorted['first'] = df_sorted['time_diff'].isna()
first_sec=df_sorted.loc[df_sorted['first']]['location'].value_counts(normalize=True)
first_sec

fruit     0.359364
dairy     0.284727
spices    0.195577
drinks    0.160332
Name: location, dtype: float64

In [14]:
next_sec=df_sorted.loc[~df_sorted['first']]['location'].value_counts(normalize=True)
next_sec

checkout    0.418097
drinks      0.164388
fruit       0.141111
dairy       0.140530
spices      0.135874
Name: location, dtype: float64

In [15]:
df_sorted.drop(columns=['time_diff', 'first', 'total_spend_time'], inplace=True)
df_sorted.set_index('timestamp', inplace=True)
df_resampled=df_sorted.groupby('customer_no').resample('1T').ffill()
df_resampled.drop(columns=['customer_no'], inplace=True)
df_resampled.head()

location
customer_no timestamp                    
1           2019-09-02 07:03:00     dairy
            2019-09-02 07:04:00     dairy
            2019-09-02 07:05:00  checkout
2           2019-09-02 07:03:00     dairy
            2019-09-02 07:04:00     dairy

In [16]:
df_resampled['next_location'] = df_resampled.groupby('customer_no')['location'].shift(-1)
df_resampled['next_location'].fillna(value='checkout', inplace=True)
df_resampled

location next_location
customer_no timestamp                                  
1           2019-09-02 07:03:00     dairy         dairy
            2019-09-02 07:04:00     dairy      checkout
            2019-09-02 07:05:00  checkout      checkout
2           2019-09-02 07:03:00     dairy         dairy
            2019-09-02 07:04:00     dairy         dairy
...                                   ...           ...
1444        2019-09-02 21:48:00    spices      checkout
            2019-09-02 21:49:00  checkout      checkout
1445        2019-09-02 21:49:00     dairy      checkout
1446        2019-09-02 21:50:00     dairy      checkout
1447        2019-09-02 21:50:00     fruit      checkout

[10765 rows x 2 columns]

### Transition Matrix

In [17]:
transition_matrix_monday = pd.crosstab(df_resampled['location'], df_resampled['next_location'], normalize='index')
transition_matrix_monday

next_location,checkout,dairy,drinks,fruit,spices
location,,,,,
checkout,1.000000,0.000000,0.000000,0.000000,0.000000
dairy,0.089843,0.743920,0.062375,0.051502,0.052361
drinks,0.208109,0.011236,0.610650,0.090865,0.079140
fruit,0.206411,0.087959,0.050821,0.607115,0.047694
spices,0.145765,0.191368,0.176710,0.096906,0.389251


In [ ]:
import pygraphviz as pgv

locations = ['checkout', 'dairy', 'drinks', 'fruit', 'spices']

# init the graph
G = pgv.AGraph(strict=False,directed=True)

# loop over all pairs of states
for state_from in locations:
    for state_to in locations:
        # get the transition probability
        proba = transition_matrix_monday.loc[state_from, state_to]
        # draw into the graph if the probability is larger zero
        if proba > 0:
            G.add_edge(state_from, state_to, label=np.round(proba, 2))

# write the graph to hard drive            
G.draw('transition_monday.png', prog='dot')